# Szövegklasszifikációs feladat

Ebben a modulban egy egyszerű szövegklasszifikációs feladattal kezdünk, amely az **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** adathalmazon alapul: hírcímeket fogunk osztályozni 4 kategória egyikébe: Világ, Sport, Üzlet és Tudomány/Technológia.

## Az adathalmaz

Az adathalmaz betöltéséhez a **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API-t fogjuk használni.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Most már hozzáférhetünk az adathalmaz képzési és tesztelési részeihez a `dataset['train']` és a `dataset['test']` használatával:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Nyomtassuk ki az adatállományunk első 10 új címsorát:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Szövegvektorozás

Most a szöveget **számokká** kell alakítanunk, amelyeket tenzorként lehet ábrázolni. Ha szószintű reprezentációt szeretnénk, két dolgot kell tennünk:

* Használjunk egy **tokenizálót**, hogy a szöveget **tokenekre** bontsuk.
* Építsünk egy **szókincset** ezekből a tokenekből.

### A szókincs méretének korlátozása

Az AG News adathalmaz példájában a szókincs mérete meglehetősen nagy, több mint 100 ezer szó. Általánosságban elmondható, hogy nincs szükségünk azokra a szavakra, amelyek ritkán fordulnak elő a szövegben — csak néhány mondatban szerepelnek, és a modell nem fog tanulni belőlük. Ezért van értelme a szókincs méretét egy kisebb számra korlátozni, amit a vektorizáló konstruktorának egy argumentumával adhatunk meg:

Mindkét lépést a **TextVectorization** réteg segítségével kezelhetjük. Hozzuk létre a vektorizáló objektumot, majd hívjuk meg az `adapt` metódust, hogy végigmenjünk az összes szövegen és felépítsük a szókincset:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Megjegyzés**: Csak az egész adatállomány egy részét használjuk a szókincs felépítéséhez. Ezt azért tesszük, hogy felgyorsítsuk a végrehajtási időt, és ne kelljen sokat várnod. Ugyanakkor vállaljuk annak kockázatát, hogy az egész adatállományból néhány szó nem kerül be a szókincsbe, és így figyelmen kívül marad az edzés során. Ezért az egész szókincs méretének használata és az adatállomány teljes átfutása az `adapt` során növelhetné a végső pontosságot, de nem jelentős mértékben.

Most már hozzáférhetünk a tényleges szókincshez:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


A vektorizáló segítségével könnyedén kódolhatunk bármilyen szöveget számok halmazává:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Szövegábrázolás szótásképzés módszerével

Mivel a szavak jelentést hordoznak, néha egy szöveg jelentését pusztán az egyes szavak alapján is megérthetjük, függetlenül attól, hogy milyen sorrendben szerepelnek a mondatban. Például hírek osztályozásakor a *időjárás* és *hó* szavak valószínűleg *időjárás-előrejelzésre* utalnak, míg a *részvények* és *dollár* szavak inkább *pénzügyi hírekhez* kapcsolódnak.

A **szótásképzés** (BoW) vektori ábrázolás a legegyszerűbben érthető hagyományos vektori ábrázolás. Minden szó egy vektor indexhez van kötve, és a vektor elemei azt mutatják, hogy egy adott dokumentumban hányszor fordul elő az adott szó.

![Kép, amely bemutatja, hogyan van ábrázolva a szótásképzés vektori reprezentációja a memóriában.](../../../../../translated_images/hu/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Megjegyzés**: A BoW-t úgy is elképzelhetjük, mint az egyes szavak egy-egy one-hot kódolt vektorának összegét a szövegben.

Az alábbiakban egy példa látható arra, hogyan lehet szótásképzés ábrázolást létrehozni a Scikit Learn python könyvtár segítségével:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

A fentebb definiált Keras vektorizálót is használhatjuk, amely minden szószámot egy one-hot kódolássá alakít, majd ezeket a vektorokat összeadja:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Megjegyzés**: Lehet, hogy meglepődsz, hogy az eredmény eltér az előző példától. Ennek az az oka, hogy a Keras példában a vektor hossza megfelel a szókincs méretének, amelyet az egész AG News adathalmazból építettünk fel, míg a Scikit Learn példában a szókincset a mintaszövegből építettük fel menet közben.


## A BoW osztályozó betanítása

Most, hogy megtanultuk, hogyan készítsük el a szövegünk bag-of-words reprezentációját, tanítsunk be egy osztályozót, amely ezt használja. Először is, az adatainkat bag-of-words reprezentációvá kell alakítanunk. Ezt a `map` függvény segítségével a következő módon érhetjük el:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Most definiáljunk egy egyszerű osztályozó neurális hálózatot, amely egy lineáris réteget tartalmaz. A bemenet mérete `vocab_size`, a kimenet mérete pedig a kategóriák számának felel meg (4). Mivel egy osztályozási feladatot oldunk meg, a végső aktivációs függvény **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Mivel négy osztályunk van, a 80% feletti pontosság jó eredménynek számít.

## Osztályozó tanítása egyetlen hálózatként

Mivel a vektorizáló is egy Keras réteg, definiálhatunk egy hálózatot, amely tartalmazza azt, és végponttól végpontig taníthatjuk. Így nem szükséges a `map` használatával vektorizálni az adathalmazt, egyszerűen átadhatjuk az eredeti adathalmazt a hálózat bemenetének.

> **Megjegyzés**: Az adathalmazunkon továbbra is alkalmaznunk kell a map-eket, hogy a mezőket (mint például `title`, `description` és `label`) szótárakból tuple-ökké alakítsuk. Azonban, ha az adatokat lemezről töltjük be, már eleve létrehozhatunk egy adathalmazt a szükséges struktúrával.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigramok, trigramok és n-gramok

A bag-of-words megközelítés egyik korlátja, hogy bizonyos szavak több szóból álló kifejezések részei lehetnek. Például a "hot dog" kifejezés teljesen más jelentéssel bír, mint a "hot" és "dog" szavak külön-külön más kontextusokban. Ha a "hot" és "dog" szavakat mindig ugyanazokkal a vektorokkal ábrázoljuk, az összezavarhatja a modellünket.

Ennek kezelésére gyakran használnak **n-gram reprezentációkat** dokumentumosztályozási módszerekben, ahol az egyes szavak, kétszavas vagy háromszavas kifejezések gyakorisága hasznos jellemző a klasszifikátorok tanításához. Például bigram reprezentációk esetén az eredeti szavak mellett az összes szópárt is hozzáadjuk a szókincshez.

Az alábbiakban egy példa látható arra, hogyan lehet bigram bag-of-words reprezentációt generálni a Scikit Learn segítségével:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Az n-gram megközelítés fő hátránya, hogy a szókincs mérete rendkívül gyorsan növekedni kezd. A gyakorlatban szükséges az n-gram reprezentációt egy dimenziócsökkentési technikával, például *beágyazásokkal* kombinálni, amelyről a következő egységben fogunk beszélni.

Ahhoz, hogy n-gram reprezentációt használjunk az **AG News** adathalmazunkban, meg kell adnunk az `ngrams` paramétert a `TextVectorization` konstruktorunknak. Egy bigram szókincs hossza **jelentősen nagyobb**, esetünkben több mint 1,3 millió token! Ezért érdemes a bigram tokeneket is ésszerű számra korlátozni.

Ugyanazt a kódot használhatnánk, mint fentebb a klasszifikátor betanításához, azonban ez nagyon memóriaigényes lenne. A következő egységben bigram klasszifikátort fogunk tanítani beágyazások segítségével. Addig is kísérletezhetsz a bigram klasszifikátor betanításával ebben a notebookban, és megnézheted, hogy el tudsz-e érni magasabb pontosságot.


## BoW vektorok automatikus kiszámítása

A fenti példában kézzel számoltuk ki a BoW vektorokat az egyes szavak egyhot kódolásának összegzésével. Azonban a TensorFlow legújabb verziója lehetővé teszi, hogy automatikusan kiszámítsuk a BoW vektorokat, ha az `output_mode='count` paramétert átadjuk a vektorizáló konstruktorának. Ez jelentősen megkönnyíti a modellünk definiálását és tanítását:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Term frekvencia - fordított dokumentum frekvencia (TF-IDF)

A BoW reprezentációban a szavak előfordulásait ugyanazzal a technikával súlyozzák, függetlenül magától a szótól. Azonban egyértelmű, hogy a gyakori szavak, mint például *a* és *in*, sokkal kevésbé fontosak az osztályozás szempontjából, mint a speciális kifejezések. A legtöbb NLP feladatban egyes szavak relevánsabbak, mint mások.

A **TF-IDF** a **term frekvencia - fordított dokumentum frekvencia** rövidítése. Ez a bag-of-words egy változata, ahol egy szó megjelenését egy dokumentumban nem bináris 0/1 értékkel jelölik, hanem egy lebegőpontos értékkel, amely a szó előfordulási gyakoriságával van összefüggésben a korpuszban.

Formálisabban, egy $i$ szó $j$ dokumentumban vett súlya $w_{ij}$ így definiálható:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ahol
* $tf_{ij}$ az $i$ szó előfordulásainak száma $j$-ben, azaz a korábban látott BoW érték
* $N$ a gyűjteményben lévő dokumentumok száma
* $df_i$ azon dokumentumok száma, amelyek tartalmazzák az $i$ szót az egész gyűjteményben

A TF-IDF érték $w_{ij}$ arányosan növekszik azzal, hogy egy szó hányszor jelenik meg egy dokumentumban, és csökken azoknak a dokumentumoknak a számával, amelyek tartalmazzák a szót a korpuszban. Ez segít korrigálni azt a tényt, hogy egyes szavak gyakrabban fordulnak elő, mint mások. Például, ha egy szó *minden* dokumentumban megjelenik a gyűjteményben, akkor $df_i=N$, és $w_{ij}=0$, így ezek a kifejezések teljesen figyelmen kívül maradnak.

A Scikit Learn segítségével könnyedén létrehozhat TF-IDF vektorizációt szövegből:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

A Kerasban a `TextVectorization` réteg automatikusan kiszámíthatja a TF-IDF gyakoriságokat az `output_mode='tf-idf'` paraméter átadásával. Ismételjük meg a fent használt kódot, hogy megnézzük, növeli-e a pontosságot a TF-IDF használata:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Következtetés

Bár a TF-IDF reprezentációk frekvenciasúlyokat rendelnek különböző szavakhoz, nem képesek jelentést vagy sorrendet ábrázolni. Ahogy a híres nyelvész, J. R. Firth 1935-ben mondta: "Egy szó teljes jelentése mindig kontextuális, és a jelentés kontextustól független tanulmányozása nem vehető komolyan." A kurzus későbbi részében megtanuljuk, hogyan lehet nyelvi modellezéssel megragadni a szöveg kontextuális információit.



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
